In [7]:
import sklearn
import time
import lxml
from lxml import etree
import pymysql
import pandas as pd

In [8]:
MAX_HREF_NUM = 1000

In [9]:
def is_chinese(string):
    """
    检查整个字符串是否包含中文
    :param string: 需要检查的字符串
    :return: bool
    """
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True

    return False

In [24]:
import os
import chardet

def get_data(path, ispms):

    html_path = path
    files = os.listdir(html_path)
    test_datas = []
    a = time.time()
    
    new_files = []
    if ispms == 1:
        db = pymysql.connect(
            host='1.15.220.155',
            user='test',
            password='991125',
            database='spider',
            charset='utf8'
        )
        c = db.cursor()
        c.execute('select distinct(domain) from pms')
        results = c.fetchall()
        results = [i[0] for i in results]
        for file in files:
            if file.split('.html')[0] in results:
                new_files.append(file)

        files = new_files
    for index, file in enumerate(files, 1):
        if '.html' not in file:
            continue
        if index % 100 == 0:
            print('{} sites have been processed correctly.'.format(index))
        file_domain = file.split('.html')[0]
        file_path = os.path.join(html_path, file)
        html_data = []
        with open(file_path, 'rb') as f:
            results = f.read()
            code = chardet.detect(results)
            try:
                results = results.decode(code['encoding'])
            except:
                try:
                    results = results.decode('GB18030')
                except:
                    try:
                        results = results.decode('utf8', errors='ignore')
                    except:
                        print('file: {}, code: {}'.format(file, code))
                        print('Not valid encode.')
                        continue
            html = etree.HTML(results)
            if html is None:
                continue
#             html_data = html.xpath('//*[@href]/text()')
            html_data = html.xpath('//*[name(.)!="style" and name(.)!="script"]/text()')
            if len(html_data) >= MAX_HREF_NUM:  # 只选取前多少个带有链接的文本
                html_data = html_data[:MAX_HREF_NUM]
            content_data = html.xpath('//*[@content]/@content')
            content_data_valid = []
            for content in content_data:
                if is_chinese(content):
                    content_data_valid.append(content)
            html_data += content_data_valid
        if html_data == []:
            continue


        one_item = ''
        code = ''
        for item in html_data:
            item = item.strip()
            one_item += ' '
            one_item += item
        movie_turple = (file_domain, ispms, one_item)
        test_datas.append(movie_turple)
    b = time.time()
    print('time used: {}'.format(b-a))
    
    return test_datas

In [25]:
# get_data('../splashHtmlCrawler/html/pms', 1)

In [26]:
pms_path = '../splashHtmlCrawler/html/pms'
not_pms_path = '../splashHtmlCrawler/html/notpms'
pms_data = get_data(pms_path, 1)
not_pms_data = get_data(not_pms_path, 0)

100 sites have been processed correctly.
200 sites have been processed correctly.
300 sites have been processed correctly.
400 sites have been processed correctly.
500 sites have been processed correctly.
600 sites have been processed correctly.
700 sites have been processed correctly.
800 sites have been processed correctly.
time used: 114.47730803489685
100 sites have been processed correctly.
200 sites have been processed correctly.
300 sites have been processed correctly.
400 sites have been processed correctly.
500 sites have been processed correctly.
600 sites have been processed correctly.
700 sites have been processed correctly.
800 sites have been processed correctly.
900 sites have been processed correctly.
1000 sites have been processed correctly.
1100 sites have been processed correctly.
1200 sites have been processed correctly.
1300 sites have been processed correctly.
1400 sites have been processed correctly.
time used: 178.03227829933167


In [27]:
all_datas = pms_data + not_pms_data

In [28]:
pd_raw_data = pd.DataFrame(all_datas)
pd_raw_data.columns = ['domain', 'target', 'text']
pd_raw_data

,domain,target,text
0,00000.host,1,资源库-资源分享论坛 ...
1,00ysw.com,1,人人影视 - 在线免费高清电影！ ...
2,100161.com,1,最新免费在线电影-神马电影网-飘花影院-青苹果影院-6080-新视觉影院-策驰影院-神马影...
3,100fyy.net,1,免费电影_免费电视剧_免费动画片在线观看 - 100分影院 ...
4,1080p.site,1,1080p.site | 精选1080p电影在线观看 ...
...,...,...,...
2358,zsb.cucn.edu.cn,0,电影学(院线管理) ...
2359,zw.liuxue86.com,0,九月最值得期待的院线电影|电影评论 ...
2360,zxwz.xjcz.gov.cn,0,网站首页 ﻿ 设为首页 收藏本站 联系我们 网...
2361,zy3.xidian.edu.cn,0,挑灯寻影|阿竹影院首映场完美结束！精彩影评与下期预告同时奉上~-西安电子科技大学竹园...


In [29]:
pd_raw_data.to_csv('movie_raw_data.csv')